In [1]:
import warnings; warnings.filterwarnings("ignore")
import os;
from sys import platform
print(os.getcwd())
if platform == "darwin":
    os.chdir('/Users/ziyingc/Dropbox/Lab/MotorCortex')
elif platform == "win32":
    os.chdir('D:\Dropbox\Lab\MotorCortex')
print(os.getcwd())
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from matplotlib.patches import Patch as patch
from matplotlib.gridspec import GridSpec
import matplotlib.cm as cm; import matplotlib.mlab as mlab
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

from scipy import stats
import read_data as rd
import behavior_track as bt
import turning_track  as tt
import time2frequency as t2f
if not os.path.exists('pic2'):
    os.makedirs('pic2')

fps = 30; fs_out = 1000; fs = 30000; T = 1/fs_out;

D:\Dropbox\Lab\MotorCortex\Multi_Rats_ver2.0
D:\Dropbox\Lab\MotorCortex


In [2]:
erfdn = np.load('Rats/Rats_ephy.npy')
###################################
mode = 'FFT'
freqs = np.load('Rats/freqs_'+mode+'_'+str(fs_out)+'.npy');  x_fit = np.log10(freqs)
idx_f2 = np.logical_and(freqs>=7.5, freqs<=30)
for rfdn in erfdn:
    epn    = np.load(rfdn+'tracking'+'/epn_ephy.npy');
    subset = np.load(rfdn+'tracking'+'/subset_ephy_li.npy');
    print(rfdn, len(epn), subset)

    ii = 0
    for k in epn:
        fdn = rfdn + k + '/'
        idx_clean_fft  = np.load(fdn+'tracking/'+'idx_clean_fft.npy')
        idx_clean_fft_not = np.logical_not(idx_clean_fft)
        
        txt_cwt  = np.round(np.sum(idx_clean_fft)/len(idx_clean_fft),5)
        idx_clean = idx_clean_fft
        
        y_clean_cwt  = np.ones((len(idx_clean),1))*0.5

        ts = np.load(fdn+'tracking/'+'ts_f.npy'); XY = np.load(fdn+'tracking/'+'XY_f.npy')

        ii = ii + 1

        fig = plt.figure(1, figsize=(20, 8))
        gs = GridSpec(8, 8);
        ax_idx = gs[6:8, 0:7];ax = fig.add_subplot(ax_idx);
        ax.plot(ts[idx_clean_fft],    y_clean_cwt[idx_clean_fft],    'b.',ms = 0.05)
        ax.plot(ts[idx_clean_fft_not],y_clean_cwt[idx_clean_fft_not],'r*',ms = 1, label = 'fft'+str(txt_cwt))

        ax.legend(loc = 'upper left', fontsize = 8)
        ax.set_xlim(ts[[0,-1]]);ax.set_ylim([0,5])
        idx_t = np.arange(len(ts))
        idx_clean_th = idx_t>-1
        l_t = len(ts)
        
        jj = 0
        for i in subset:
            chn   = str(i//10)+str(i%10)
            Sxx_f = np.load(fdn + mode+'_'+str(fs_out)+'/ch.'+chn+'.fps.npy').T
            ax_idx = gs[jj*3:(jj+1)*3,0:7];ax = fig.add_subplot(ax_idx);
            Sxx_f[:,idx_clean_fft_not] = 0
            im = ax.pcolormesh(ts, freqs[idx_f2], Sxx_f[idx_f2, :], 
                               vmin = 0, vmax = 500,
                               cmap=cm.jet);
#             fig.colorbar(im, ax=ax);

            ax.set_ylabel('Hz', fontsize = 15);
            ax.tick_params(labelsize=12);
            ax.set_title(chn, fontsize = 15)
            jj=jj+1

        plt.savefig(rfdn+'pic2/'+k+'_scalogram_fft.png')
        plt.clf()

Rats/Rat_Swashbuckler/ 21 [0 4]
Rats/Rat_Outlaw/ 61 [ 5 13]
Rats/Rat_Drifter/ 52 [20 28]
Rats/Rat_Unknown/ 15 [20  4]
Rats/Rat_Quartus/ 29 [ 4 28]
Rats/Rat_Lenny/ 11 [ 5 12]
Rats/Rat_Astro/ 17 [ 5 12]


<Figure size 1440x576 with 0 Axes>

In [3]:
erfdn = [
    'Rats/Rat_Astro/',
    'Rats/Rat_Drifter/',
    'Rats/Rat_Lenny/',
    'Rats/Rat_Quartus/',
    'Rats/Rat_Unknown/',
    'Rats/Rat_Outlaw/',
    'Rats/Rat_Swashbuckler/',
]

In [2]:

erfdn = np.load('Rats/Rats_ephy.npy')
f = np.load('Rats/freqs_FFT_'+str(fs_out)+'.npy');

j = 0
for rfdn in erfdn:
    epn    = np.load(rfdn+'tracking'+'/epn_ephy.npy');
    subset = np.load(rfdn+'tracking'+'/subset_ephy.npy');
    print(subset)
    rc  = int(np.sqrt(len(epn)))+1
    fig = plt.figure(1, figsize=(rc*4, rc*4))
    gs  = GridSpec(rc , rc);jj = 0
    for k in epn:
        fdn = rfdn + k + '/';
        idx_clean_f = np.load(fdn+'tracking/'+'idx_clean_f.npy')
        idx_clean   = np.logical_and(idx_clean_f, idx_idol_f)        
        ax_idx = gs[jj];ax = fig.add_subplot(ax_idx);  
        ii = 0
        for i in subset: 
            chn = str(i//10)+str(i%10);

            Y_f = np.load(fdn + 'FFT_'+str(fs_out)+'/ch.'+chn+'.fps.npy');
            psd = np.mean(Y_f[idx_clean,:], axis=0)
            ax.plot(f, psd, label = chn, lw=0.5);   ii = ii+1
            ax.set_ylim([1, 80]); ax.set_xlim([3,55])

        ax.tick_params(labelsize=8);
        ax.set_title(k[10:21], fontsize = 10);
        jj=jj+1
    ax.legend()
    plt.savefig('pic2/BBB_'+rfdn[9:len(rfdn)-1]+'.png')
    plt.clf()


[0 1 4 5]


NameError: name 'idx_idol_f' is not defined

<Figure size 1152x1152 with 0 Axes>

In [8]:
txt_line = ['b' , 'r']
txt_tn   = ['lesion','intact']
fl = 20; fh = 50
erfdn = np.load('Rats/Rats_ephy.npy')
f = np.load('Rats/freqs_FFT_'+str(fs_out)+'.npy');     print(f[0:10])
j = 0
for rfdn in erfdn:
#     epn    = np.load(rfdn+'tracking'+'/epn_LT.npy');
#     epn    = np.load(rfdn+'tracking'+'/epn_turn_elec_post.npy');
    epn    = np.load(rfdn+'tracking'+'/epn_ephy.npy');
    subset = np.load(rfdn+'tracking'+'/subset_ephy_li.npy');

    rc = int(np.sqrt(len(epn)))+1
    fig = plt.figure(1, figsize=(rc*3, rc*3))
    gs  = GridSpec(rc , rc);jj = 0

    for k in epn:
        fdn = rfdn + k + '/'
        idx_clean_f = np.load(fdn+'tracking/'+'idx_clean_f.npy')
        idx_clean_fft = np.load(fdn+'tracking/'+'idx_clean_fft.npy')
        idx_clean = np.logical_and(idx_clean_f,   idx_clean_fft)
 
        ax_idx = gs[jj];ax = fig.add_subplot(ax_idx);
        ii = 0
        for i in subset: 
            chn = str(i//10)+str(i%10);   
            Y_f = np.load(fdn + 'FFT_'+str(fs_out)+'/ch.'+chn+'.fps.npy')
            psd = np.mean(Y_f[idx_clean,:],axis=0);
#             psd = np.mean(Y_f[idx_clean,:],axis=0)
            ax.plot(f, psd, label = chn);ii = ii+1
            ax.set_ylim([10, 100])
            ax.set_xlim([5, 60]);
#             ax.set_ylim([-0.5, 0.5]) 

        ax.tick_params(labelsize=8);
        ax.set_title(k[10:21], fontsize = 10);
        jj=jj+1
    chn0 = str(subset[0]//10)+str(subset[0]%10);
    chn1 = str(subset[1]//10)+str(subset[1]%10);
    ax.legend()
    plt.savefig('pic2/BBB_2chs_'+rfdn[9:len(rfdn)-1]+'_'+chn0+chn1+'.png')
    plt.clf()

[  2.   4.   6.   8.  10.  12.  14.  16.  18.  20.]


<Figure size 864x864 with 0 Axes>

In [11]:
txt_line = ['b' , 'r']
txt_tn = ['lesion','intact']
fl = 18; fh = 50
erfdn = np.load('Rats/Rats_ephy.npy')


mode = 'FFT'
f = np.load('Rats/freqs_'+mode+'_'+str(fs_out)+'.npy');  
for rfdn in erfdn:
    epn    = np.load(rfdn+'tracking'+'/epn_ephy.npy');
    subset = np.load(rfdn+'tracking'+'/subset_ephy_li.npy');   
#     idx_f1 = np.logical_and(f>=fl, f<=26);
#     idx_f2 = np.logical_and(f>=42, f<=fh);
#     idx_f = np.logical_or(idx_f1, idx_f2);
    idx_f = np.logical_and(f>=fl, f<=fh);
    slope_l, intercept_l, slope_i, intercept_i, intercept_li = \
    t2f.flicker_fit_fft(rfdn, epn, fs_out, subset, f, idx_f, mode)
    slope     = np.array([slope_l,     slope_i]);
    intercept = np.array([intercept_l, intercept_i]);
    
    rc = int(np.sqrt(len(epn)))+1
    fig = plt.figure(1, figsize=(rc*4, rc*4))
    gs  = GridSpec(rc , rc);jj = 0

    Psd    = np.zeros((len(epn)*2, len(f)));

    for k in epn:
        fdn = rfdn + k + '/'
        idx_clean_f = np.load(fdn+'tracking/'+'idx_clean_f.npy')
        idx_idol_f  = np.load(fdn+'tracking/'+'idx_idol_f.npy')
        idx_clean = np.logical_and(idx_clean_f, idx_idol_f)
#         idx_clean_cwt = np.load(fdn+'tracking/'+'idx_clean_cwt.npy')
#         idx_clean = np.logical_and(idx_clean,   idx_clean_cwt)
        ax_idx = gs[jj];
        ax     = fig.add_subplot(ax_idx);  
        ii = 0
        for i in subset: 
            chn = str(i//10)+str(i%10);   
            Y_f = np.load(fdn + 'FFT_'+str(fs_out)+'/ch.'+chn+'.fps.npy')
            psd = np.mean(Y_f[idx_clean,:],axis=0)
            ax.plot(np.log10(f), np.log10(psd), txt_line[ii], lw=1,label = chn)   
#             ax.set_ylim([1.0, 1.7]); ax.set_xlim([np.log10(fl), np.log10(fh)])
            ax.set_ylim([0.5, 2]); ax.set_xlim([0.5, 2])
#             ax.set_xlim([1.2, 1.9])
            Psd[jj*2+ii,:] = psd
            ii = ii+1
        ax.tick_params(labelsize=8);
        ax.set_title(k[10:21], fontsize = 8);
        jj=jj+1

    jj = 0
    x_fit = np.log10(f[idx_f])
    for k in epn:
        fdn = rfdn + k + '/'
        ax_idx = gs[jj];ax = fig.add_subplot(ax_idx);  
        y_fit = x_fit*slope_l[jj] + intercept_l[jj]; 
        ax.plot(x_fit, y_fit, txt_line[0]+'.', ms = 1);
        y_fit = x_fit*slope_i[jj] + intercept_i[jj]; 
        ax.plot(x_fit, y_fit, txt_line[1]+'.', ms = 1);
        jj=jj+1
    ax.legend()
    plt.savefig('pic2/BBB_2chs_'+rfdn[9:len(rfdn)-1]+'_fit.png')
    plt.clf()

<Figure size 2304x2304 with 0 Axes>

In [18]:
# erfdn = np.load('Rats/Rats_LT.npy')
erfdn = np.load('Rats/Rats_Turn.npy')
txt_line = ['b' , 'r']
txt_tn = ['lesion','intact']
f = np.load('freqs_fft_'+str(fs_out)+'.npy');   x_fit = np.log10(f)
fl = 14; fh = 54
for rfdn in erfdn:
#     epn    = np.load(rfdn+'tracking'+'/epn_LT_post.npy');
    epn    = np.load(rfdn+'tracking'+'/epn_turn_elec_post.npy');
    subset = np.load(rfdn+'tracking'+'/subset_ephy_li.npy');
#     idx_f1 = np.logical_and(f>=fl, f<=26);
#     idx_f2 = np.logical_and(f>=42, f<=fh);
#     idx_f = np.logical_or(idx_f1, idx_f2);
    idx_f = np.logical_and(f>=fl, f<=fh);
    slope_l, intercept_l, slope_i, intercept_i, intercept_li = \
    t2f.flicker_fit_fft(rfdn, epn, fs_out, subset, f, idx_f)
    
    slope     = np.array([slope_l,     slope_i]);
    intercept = np.array([intercept_l, intercept_i]);
    print(slope.shape, intercept.shape)
    fig = plt.figure(1, figsize=(12,8))
    gs = GridSpec(2, 3);
    fig.suptitle(rfdn+', Power spectrum of '+str(len(epn))+' sessions', fontsize=20)

    jj = 0
    for k in epn:
        fdn = rfdn + k + '/'
        idx_clean_f = np.load(fdn+'tracking/'+'idx_clean_f.npy')
        idx_idol_f  = np.load(fdn+'tracking/'+'idx_idol_f.npy')
        idx_clean   = np.logical_and(idx_clean_f, idx_idol_f)
        idx_clean_f_cwt = np.load(fdn+'tracking/'+'idx_clean_cwt.npy')
        idx_clean    = np.logical_and(idx_clean, idx_clean_f_cwt)
        
        i = subset[1]
        chn    = str(i//10)+str(i%10)
        Y_f = np.load(fdn + 'FFT_'+str(fs_out)+'/ch.'+chn+'.fps.npy').T
        psd_mean = np.mean(Y_f[:,idx_clean],axis=1);
        psd_std = np.std(Y_f[:,idx_clean],axis=1);

        psd_m = np.zeros([2, len(f)])
        psd_f = np.zeros([2, len(f)])
        psd_z = np.zeros([2, len(f)])
        ii = 0
        for i in subset:
            
            chn    = str(i//10)+str(i%10)
            Y_f = np.load(fdn + 'FFT_'+str(fs_out)+'/ch.'+chn+'.fps.npy').T         

            psd = np.mean(Y_f[:,idx_clean],axis=1);
            ax_idx = gs[0, 0];ax = fig.add_subplot(ax_idx);
            ax.plot(f, psd, txt_line[ii], lw = 0.8);
            psd_m[ii,:] = psd
            
            y_fit = 10**(x_fit*slope[ii,jj] + intercept[ii,jj]);
            Y_f_fit = (Y_f.T/y_fit).T
            psd = np.mean(Y_f_fit[:,idx_clean],axis=1);
            ax_idx = gs[0, 1];ax = fig.add_subplot(ax_idx);
            ax.plot(f, psd, txt_line[ii], lw = 0.8);
            psd_f[ii,:] = psd

            Y_f_z = ((Y_f.T-psd_mean)/psd_std).T
            psd = np.mean(Y_f_z[:,idx_clean],axis=1);
            ax_idx = gs[0, 2];ax = fig.add_subplot(ax_idx);
            ax.plot(f, psd, txt_line[ii], lw = 0.8);
            psd_z[ii,:] = psd

            ii = ii+1

        jj = jj+1

        ax_idx = gs[1, 1];ax = fig.add_subplot(ax_idx);
        ax.plot(f, psd_f[0,:]-psd_f[1,:], txt_line[0], lw = 0.8);


    ax_idx = gs[0,0];ax = fig.add_subplot(ax_idx);ax.set_xlabel('Hz');  ax.legend();
    ax.set_title('Power Spectrum', fontsize=7);
    ax.set_xlim([fl, fh]); ax.set_ylim([0,100])
    ax_idx = gs[0,1];ax = fig.add_subplot(ax_idx);ax.set_xlabel('Hz');
    ax.set_title('Power Spectrum, Normalized to 1/f noise', fontsize=7);
    ax.set_xlim([fl, fh]); ax.set_ylim([0, 4])
    
    ax_idx = gs[1,1];ax = fig.add_subplot(ax_idx);ax.set_xlabel('Hz');
    ax.set_title('Power Spectrum, Normalized to 1/f noise', fontsize=7);
    ax.set_xlim([fl, fh]); ax.set_ylim([-0.2, 0.5])
    
    ax_idx = gs[0,2];ax = fig.add_subplot(ax_idx);ax.set_xlabel('Hz');
    ax.set_title('Power Spectrum, Normalized (x-m)/sigma)', fontsize=7);
    ax.set_xlim([fl, fh]); ax.set_ylim([-0.5,1.5])
    plt.savefig('pic2/BBB_fit_'+rfdn[9:len(rfdn)-1]+'.png')
    plt.clf()

(2, 52) (2, 52)


No handles with labels found to put in legend.


(2, 5) (2, 5)


No handles with labels found to put in legend.


(2, 15) (2, 15)


No handles with labels found to put in legend.


(2, 38) (2, 38)


No handles with labels found to put in legend.


<Figure size 864x576 with 0 Axes>

In [11]:

import shutil
# erfdn = np.load('Rats/Rats_ephy.npy')
# erfdn = np.load('Rats/Rats_LT.npy')
# erfdn = np.load('Rats/Rats_Turn.npy')
# erfdn = np.load('Rats/Rats_Turn_ephy.npy')
for rfdn in erfdn:
    epn    = np.load(rfdn+'tracking'+'/epn_ephy_li_post.npy');
#     epn    = np.load(rfdn+'tracking'+'/epn_turn_elec_post.npy');
#     epn    = np.load(rfdn+'tracking'+'/epn_LT_post.npy');
#     epn    = np.load(rfdn+'tracking'+'/epn_track.npy');
#     subset = np.load(rfdn+'tracking'+'/subset_ephy_li.npy');
    for k in epn:
        fdn = rfdn + k + '/'
#         try:
#             shutil.rmtree(fdn + 'FFT_'+str(500))
#         except:
#             print('no beta')
            

Rats/Rat_Outlaw/ 62 [ 5 13]
Rats/Rat_Swashbuckler/ 23 [0 4]
Rats/Rat_Drifter/ 66 [20 28]
Rats/Rat_Unknown/ 16 [20  4]
Rats/Rat_Quartus/ 29 [ 4 28]
Rats/Rat_Astro/ 17 [ 5 12]
Rats/Rat_Lenny/ 11 [ 5 12]


<Figure size 1440x576 with 0 Axes>

In [3]:
# erfdn = np.load('Rats/Rats_ephy.npy')
# erfdn = np.load('Rats/Rats_LT.npy')
# erfdn = np.load('Rats/Rats_Turn.npy')
# erfdn = np.load('Rats/Rats_Turn_ephy.npy')
erfdn = np.load('Rats/Rats_LT_ephy.npy')

# erfdn = [
#     'Rats/Rat_Swashbuckler/',
# ]
###################################
f = np.load('freqs_fft_'+str(fs_out)+'.npy');   x_fit = np.log10(f)
fl = 12; fh = 56
idx_f1 = np.logical_and(f>=fl, f<=22);
idx_f2 = np.logical_and(f>=46, f<=fh);
idx_f = np.logical_or(idx_f1, idx_f2);

idx_band  = np.logical_and(f>=5, f<=55)
for rfdn in erfdn:
#     epn    = np.load(rfdn+'tracking'+'/epn_ephy_li_post.npy');
#     epn    = np.load(rfdn+'tracking'+'/epn_turn_elec_post.npy');
    epn    = np.load(rfdn+'tracking'+'/epn_LT_post.npy');
    subset = np.load(rfdn+'tracking'+'/subset_ephy_li.npy');

    slope_l, intercept_l, slope_i, intercept_i, intercept_li = \
    t2f.flicker_fit_fft(rfdn, epn, fs_out, subset, f, idx_f)
    slope     = np.array([slope_l,     slope_i]);
    intercept = np.array([intercept_l, intercept_i]);

    kk = 0
    for k in epn:
        fdn = rfdn + k + '/'
        idx_clean_cwt  = np.load(fdn+'tracking/'+'idx_clean_cwt.npy')
        idx_idol       = np.load(fdn+'tracking/'+'idx_idol_f.npy')
        idx_clean_cwt_not = np.logical_not(idx_clean_cwt)
        idx_idol_not      = np.logical_not(idx_idol)
        
        txt_cwt  = np.round(np.sum(idx_clean_cwt)/len(idx_clean_cwt),5)
        txt_idle = np.round(np.sum(idx_idol)/len(idx_idol),5)
        idx_clean = np.logical_and(idx_clean_cwt, idx_idol)
        y_clean_cwt  = np.ones((len(idx_clean),1))*0.5
        y_clean_idle = np.ones((len(idx_clean),1))*1
        timestamps = np.load(fdn+'tracking/'+'ts_f.npy')
        XY         = np.load(fdn+'tracking/'+'XY_f.npy')

        
        i = subset[1];   chn = str(i//10)+str(i%10)
        Sxx_f = np.load(fdn + 'CWT_'+str(fs_out)+'/ch.'+chn+'.fps.npy')
        Sxx_f_m = np.mean(Sxx_f[idx_clean,:],axis=0);
        Sxx_f_s = np.std(Sxx_f[idx_clean,:], axis=0);
#         print(Sxx_f.shape, Sxx_f_m.shape, Sxx_f_s.shape)


        fig = plt.figure(1, figsize=(25, 10))
        fig.suptitle(k, fontsize=15)
        gs = GridSpec(8, 8);
        ax_idx = gs[6:8, 0:7];ax = fig.add_subplot(ax_idx);
        ax.plot(timestamps[idx_clean_cwt],    y_clean_cwt[idx_clean_cwt],    'b.',ms = 0.05)
        ax.plot(timestamps[idx_clean_cwt_not],y_clean_cwt[idx_clean_cwt_not],'r*',ms = 1, label = 'cwt'+str(txt_cwt))
        ax.plot(timestamps[idx_idol],    y_clean_idle[idx_idol],    'b.',ms = 0.05)
        ax.plot(timestamps[idx_idol_not],y_clean_idle[idx_idol_not],'k*',ms = 1, label = 'idle'+str(txt_idle))
        ax.legend(loc = 'upper left', fontsize = 10)
        ax.set_xlim(timestamps[[0,-1]]);ax.set_ylim([0,3])
        idx_t = np.arange(len(timestamps))
        idx_clean_th = idx_t>-1
        l_t = len(timestamps)
        ii = 0
        for i in subset:
            chn    = str(i//10)+str(i%10)
            Y_f = np.load(fdn + 'FFT_'+str(fs_out)+'/ch.'+chn+'.fps.npy').T
            y_fit = 10**(x_fit*slope[ii,kk] + intercept[ii,kk]);     
            Sxx_f = (Y_f.T/y_fit).T
#         for i in subset:
#             chn   = str(i//10)+str(i%10)
#             Sxx_f = np.load(fdn + 'CWT_'+str(fs_out)+'/ch.'+chn+'.fps.npy')
#             Sxx_f = ((Sxx_f - Sxx_f_m)/Sxx_f_s).T
            
            ax_idx = gs[ii*3:(ii+1)*3,0:7];ax = fig.add_subplot(ax_idx);  
            im = ax.pcolormesh(timestamps, f[idx_band], Sxx_f[idx_band, :], 
                               vmin = 0, vmax = 20,
                               cmap=cm.jet);
#             fig.colorbar(im, ax=ax);

            ax.set_ylabel('Hz'); 
            ax.tick_params(labelsize=8);
            ax.set_title(chn, fontsize = 15)

            ii=ii+1

        plt.savefig(rfdn+'pic2/'+k+'_scalogram_fit.png')
        plt.clf()

<Figure size 1800x720 with 0 Axes>

In [10]:
txt_line = ['b' , 'r']
txt_tn = ['lesion','intact']
fl = 14; fh = 54
fig = plt.figure(1, figsize=(15,5*len(erfdn)))
gs = GridSpec(len(erfdn), 3);
kk = 0
for rfdn in erfdn:
    epn    = np.load(rfdn+'tracking'+'/epn_ephy_li_post.npy');
    subset = np.load(rfdn+'tracking'+'/subset_ephy_li.npy');
    f      = np.load(rfdn +'f_fft.npy');    x_fit = np.log10(f)
#     idx_f1 = np.logical_and(f>=fl, f<=26);
#     idx_f2 = np.logical_and(f>=42, f<=fh);
#     idx_f = np.logical_or(idx_f1, idx_f2);
    idx_f = np.logical_and(f>=fl, f<=fh);
    slope_l, intercept_l, slope_i, intercept_i, intercept_li = \
    t2f.flicker_fit_fft(rfdn, epn, fs_out, subset, f, idx_f)
    
    slope     = np.array([slope_l,     slope_i]);
    intercept = np.array([intercept_l, intercept_i]);
    print(slope.shape, intercept.shape)
#     fig = plt.figure(1, figsize=(12,8))
#     gs = GridSpec(2, 3);
#     fig.suptitle(rfdn+', Power spectrum of '+str(len(epn))+' sessions', fontsize=20)

    jj = 0
    for k in epn:
        fdn = rfdn + k + '/'
        idx_clean_f = np.load(fdn+'tracking/'+'idx_clean_f.npy')
        idx_idol_f  = np.load(fdn+'tracking/'+'idx_idol_f.npy')
        idx_clean = np.logical_and(idx_clean_f, idx_idol_f)
        idx_clean_f_cwt = np.load(fdn+'tracking/'+'idx_clean_cwt.npy')
        idx_clean = np.logical_and(idx_clean, idx_clean_f_cwt)
        
        ii = 0

#         y_fit = 10**(x_fit*slope_i[jj] + intercept_i[jj]);
        i = subset[1]; chn    = str(i//10)+str(i%10)
        Y_f = np.load(fdn + 'FFT_'+str(fs_out)+'/ch.'+chn+'.P_ft.npy').T
        psd_mean = np.mean(Y_f[:,idx_clean],axis=1);
        psd_std = np.std(Y_f[:,idx_clean],axis=1);
        
        psd_m = np.zeros([2, len(f)])
        psd_f = np.zeros([2, len(f)])
        psd_z = np.zeros([2, len(f)])

        for i in subset:
            
            chn    = str(i//10)+str(i%10)
            Y_f = np.load(fdn + 'FFT_'+str(fs_out)+'/ch.'+chn+'.P_ft.npy').T         
            
            
            psd = np.mean(Y_f[:,idx_clean],axis=1);
            ax_idx = gs[kk,0];ax = fig.add_subplot(ax_idx);
            ax.plot(f, psd, txt_line[ii], lw = 0.8);
            psd_m[ii,:] = psd
            
            y_fit = 10**(x_fit*slope[ii,jj] + intercept[ii,jj]);
            Y_f_fit = (Y_f.T/y_fit).T
            psd = np.mean(Y_f_fit[:,idx_clean],axis=1);
            ax_idx = gs[kk,1];ax = fig.add_subplot(ax_idx);
            ax.plot(f, psd, txt_line[ii], lw = 0.8);
            psd_f[ii,:] = psd

            Y_f_z = ((Y_f.T-psd_mean)/psd_std).T
            psd = np.mean(Y_f_z[:,idx_clean],axis=1);
            ax_idx = gs[kk,2];ax = fig.add_subplot(ax_idx);
            ax.plot(f, psd, txt_line[ii], lw = 0.8);
            psd_z[ii,:] = psd

            ii = ii+1

        jj = jj+1

    ax_idx = gs[kk,0];ax = fig.add_subplot(ax_idx);#ax.set_xlabel('Hz');  ax.legend();
    ax.set_ylabel(rfdn[9:len(rfdn)-1], fontsize=20);
    ax.set_title('Power Spectrum', fontsize=10);
    ax.set_xlim([fl, fh]); ax.set_ylim([0,100])
    ax_idx = gs[kk,1];ax = fig.add_subplot(ax_idx); 
    ax.set_title('Power Spectrum, Normalized to 1/f noise', fontsize=10);
    ax.set_xlim([fl, fh]); ax.set_ylim([0.5, 2.8])
    ax_idx = gs[kk,2];ax = fig.add_subplot(ax_idx);#ax.set_xlabel('Hz');
    ax.set_title('Power Spectrum, Normalized (x-m)/sigma)', fontsize=10);
    ax.set_xlim([fl, fh]); ax.set_ylim([-0.5,1.5])
    kk = kk + 1
plt.savefig('pic2/BBB_fit.png')
plt.clf()

(2, 11) (2, 11)


<Figure size 1080x360 with 0 Axes>